<a href="https://colab.research.google.com/github/ejrtks1020/Final_preparation_for_Bigdata_exam/blob/main/%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D%EA%B8%B0%EC%82%AC_final_practice_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

df = pd.read_excel('data1.xlsx')

In [ ]:
df.head()

,날짜,SKU ID,SKU 명,바코드,발주가능상태,입고수량,쿠팡출고수량,쿠팡현재재고수량,자재번호,자재명,카테고리,입수,출고수량,재고수량
0,2019-05-16,29349,리큐진한겔일반 3LNEW,8801046893111,발주가능,384,287,5788,6077374,19리큐 진한겔 알카2.7용기(일),세탁세제,4,71.750000,80.668990
1,2019-05-16,29350,리큐 진한겔드럼 3L NEW,8801046893128,발주가능,0,235,2772,6077394,19리큐 진한겔 알카2.7용기(드),세탁세제,4,58.750000,47.182979
2,2019-05-16,29351,울샴푸아웃도어 800ml 용기,8801046878934,발주가능,0,2,35,6077406,19울샴푸아웃도 800ml(리뉴얼),세탁세제,9,0.222222,157.500000
3,2019-05-16,29352,울샴푸아웃도어 1L 일자형 리필,8801046878941,발주가능,207,41,2083,6077404,19울샴푸아웃도 1L(리뉴얼),세탁세제,9,4.555556,457.243902
4,2019-05-16,29359,Pack 2080오리지날칫솔 5+5입_탄력모,8801046888858,발주가능,288,312,3411,5009109,182080오리지날칫솔 5+5입(탄력모),덴탈케어,8,39.000000,87.461538


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36833 entries, 0 to 36832
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   날짜        36833 non-null  object 
 1   SKU ID    36833 non-null  int64  
 2   SKU 명     36833 non-null  object 
 3   바코드       36833 non-null  object 
 4   발주가능상태    36833 non-null  object 
 5   입고수량      36833 non-null  int64  
 6   쿠팡출고수량    36833 non-null  int64  
 7   쿠팡현재재고수량  36833 non-null  int64  
 8   자재번호      36833 non-null  int64  
 9   자재명       36833 non-null  object 
 10  카테고리      36833 non-null  object 
 11  입수        36833 non-null  int64  
 12  출고수량      36833 non-null  float64
 13  재고수량      36833 non-null  float64
dtypes: float64(2), int64(6), object(6)
memory usage: 3.9+ MB


In [ ]:
df.isnull().sum()

날짜          0
SKU ID      0
SKU 명       0
바코드         0
발주가능상태      0
입고수량        0
쿠팡출고수량      0
쿠팡현재재고수량    0
자재번호        0
자재명         0
카테고리        0
입수          0
출고수량        0
재고수량        0
dtype: int64

- 1. '출고수량' 항목에 데이터에서 출고수량이 10개 이상인 데이터를 추출하여, 
     재고수량의 평균을 계산해보시오. 
     - 필터 
     - 통계량 (평균 / 중앙값 / 표준편차) / 데이터 수 

In [ ]:
################### sol ####################
cond1 = (df['출고수량'] >= 10)
df2 = df.loc[cond1]

print(df2.shape) #출고수량이 10개 이상인 데이터가 6276개
print(df2['재고수량'].mean()) #재고수량의 평균은 144,944

(6276, 14)
144.9940311993694


In [ ]:
############# My Code #####################
df[df['출고수량'] >= 10]['재고수량'].mean()

144.9940311993694

- 2. '재고수량'데이터를 Standard Scaling을 실시하여, 0.5이상의 데이터의 갯수(row)  

In [ ]:
################### sol ####################
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

scaler = StandardScaler()
scaler.fit(df[['재고수량']])
result = scaler.transform(df[['재고수량']])
df_result = pd.DataFrame(result)
cond1 = (df_result[0] >= 0.5)
df_result2 = df_result[cond1]
print(df_result2.shape)

(1791, 1)


In [ ]:
############# My Code #####################
df['재고수량_scaled'] = df['재고수량'].map(lambda x : (x - df['재고수량'].mean()) / df['재고수량'].std())
print(len(df[df['재고수량_scaled'] >= 0.5]))

1791


- 3. '입고수량' 데이터에서 가장 큰 상위 10개 데이터를 가장 작은 최솟값으로 변환하여 평균을 계산 

In [ ]:
############# Sol #########################
df_top_10 = df.sort_values('입고수량', ascending=False).head(10)
print(df_top_10.shape)
print(df_top_10['입고수량'].min())
cond1 = (df['입고수량'] >= 7287)
df.loc[cond1, '입고수량(clean)'] = df['입고수량'].min()
df.loc[~cond1, '입고수량(clean)'] = df['입고수량']
df['입고수량'].mean(), df['입고수량(clean)'].mean()

(10, 15)
7287


(56.10661634946923, 53.59441261911873)

In [ ]:
############# My Code #####################
print(df['입고수량'].mean())
indexes = df['입고수량'].sort_values(ascending = False)[:10].index
df.iloc[indexes , '입고수량'] = df['입고수량'].min()
print(df['입고수량'].mean())

56.10661634946923
53.59441261911873


- 4. data1.xlsx 데이터에서 학습데이터를 70% 추출하여, 학습데이터의 '출고수량' 표준편차 와 검증데이터의 '출고수량' 표준편차의 차 를 계산 
    (random state를 1234)

In [ ]:
############# My Code #####################
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size = 0.3, random_state=1234)
print(train['출고수량'].std() - test['출고수량'].std())


-0.3610776884744915


- 5. data1.xlsx 데이터에서 '입수'의 이상치를 모두 합한 값을 계산하라 
     (이상치는 (3사분위수 + (IQR * 1.5) )를 기준으로 함) 

In [ ]:
############# My Code #####################
q3 = df['입수'].quantile(0.75)
q1 = df['입수'].quantile(0.25)
IQR = q3 - q1
cond = q3 + IQR * 1.5

df[df['입수'] >= cond]['입수'].sum()

56795

- 6. data1.xlsx 데이터에서 '출고수량'의 이상치를 모두 합한 값을 계산하라 
    (이상치는 표준편차의 3배에 해당하는 값을 기준으로 함) 

In [ ]:
################# Sol #####################
cond1 = (df['출고수량'] >= (df['출고수량'].std() * 3))
df_outlier = df.loc[cond1]
print(df_outlier['출고수량'].sum())

92892.0111111111


In [ ]:
############# My Code #####################
cond1 = df['출고수량'].mean() + df['출고수량'].std() * 3
cond2 = df['출고수량'].mean() - df['출고수량'].std() * 3

df[(df['출고수량'] > cond1) | (df['출고수량'] < cond2)]['출고수량'].sum()

80149.27777777778

- 7. data1.xlsx에서 '재고수량'값이 500개 이상인 값을 1 / 미만을 0 데이로 변환하여 'Target' Column을 생성한 뒤, 'Target' Column에 대한 분류모델을 생성하고 평가하시오.  

In [ ]:
##################### Sol ################################
cond = (df['재고수량'] >= 500)
df.loc[cond, 'target'] =1
df.loc[~cond,'target'] = 0
print(df['target'].value_counts())

0.0    32586
1.0     4247
Name: target, dtype: int64


In [ ]:
# 0. X / Y 분할
# 1. X 범주형 데이터 문자로 변환 (One Hot / pd.get_dummies)
# 2. 학습 검증데이터 분할
# 3. 파이프라인 구축(특성공학 / 결측치 처리 Imputation + 분류학습)
# 4. 교차 검증 + Hyper Parameter Tuning 실시 -> Best Model
# 5. 평가

Y = df['target']
X = df.drop(columns = ['날짜', 'SKU ID', 'SKU 명', '바코드' ,'자재번호', '자재명', '재고수량' ,'target'])

X_dummy = pd.get_dummies(X) #범주형 데이터를 숫자로 변환

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute   import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_dummy, Y, test_size = 0.3, random_state = 1234)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((25783, 17), (11050, 17), (25783,), (11050,))

In [ ]:
pipe_list = [('impute', KNNImputer()), ('model', RandomForestClassifier())]
pipe_model = Pipeline(pipe_list)
print(pipe_model)

Pipeline(steps=[('impute', KNNImputer()), ('model', RandomForestClassifier())])


In [ ]:
hyper_parameter = { 'model__min_samples_leaf':range(5, 10),
                    'model__n_estimators' : [50, 100, 150],
                    'model__class_weight' : [None, 'balanced']}
grid_model = GridSearchCV(pipe_model, param_grid = hyper_parameter, cv = 3,
                          n_jobs = -1, scoring = 'f1')                    
grid_model.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('impute', KNNImputer()),
                                       ('model', RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'model__class_weight': [None, 'balanced'],
                         'model__min_samples_leaf': range(5, 10),
                         'model__n_estimators': [50, 100, 150]},
             scoring='f1')

In [ ]:
best_model = grid_model.best_estimator_
print(best_model)

Pipeline(steps=[('impute', KNNImputer()),
                ('model',
                 RandomForestClassifier(min_samples_leaf=5, n_estimators=150))])


In [ ]:
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

print(classification_report(y_train, y_train_pred))
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     22804
         1.0       1.00      0.96      0.98      2979

    accuracy                           1.00     25783
   macro avg       1.00      0.98      0.99     25783
weighted avg       1.00      1.00      1.00     25783

              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      9782
         1.0       0.99      0.93      0.96      1268

    accuracy                           0.99     11050
   macro avg       0.99      0.96      0.97     11050
weighted avg       0.99      0.99      0.99     11050



In [ ]:
###################### My Code ###################################
df['Target'] = df['재고수량'].map(lambda x : 1 if x >= 500 else 0)

In [ ]:
df.head(2)

,날짜,SKU ID,SKU 명,바코드,발주가능상태,입고수량,쿠팡출고수량,쿠팡현재재고수량,자재번호,자재명,카테고리,입수,출고수량,재고수량,재고수량_scaled,Target
0,2019-05-16,29349,리큐진한겔일반 3LNEW,8801046893111,발주가능,384,287,5788,6077374,19리큐 진한겔 알카2.7용기(일),세탁세제,4,71.75,80.668990,-0.146010,0
1,2019-05-16,29350,리큐 진한겔드럼 3L NEW,8801046893128,발주가능,0,235,2772,6077394,19리큐 진한겔 알카2.7용기(드),세탁세제,4,58.75,47.182979,-0.169367,0


In [ ]:
df.Target.value_counts()

0    32586
1     4247
Name: Target, dtype: int64

In [ ]:
df['자재명'].value_counts()

19리큐 진한겔 알카2.7용기(일)       242
19KS퍼퓸린스엘레강스 600ml책       168
19리큐 진한겔 1/2 2.4용기(드)     168
19리큐 진한겔 1/2 2.4용기(일)     168
18울샴푸오리지 1.8L (화)         168
                         ... 
19샤워메이트스크럽 300쿠키앤크림책        7
18 스파크 스팀100℃ 1.4kg 기획      7
18샤워메이트스크럽 300피스타치오         7
17샤워메이트필링티슈 10매브라이트닝        7
18KS걸리쉬 헤어파우더팩트             7
Name: 자재명, Length: 351, dtype: int64

In [ ]:
len(df[df['입수'] >= cond]), len(df[(df['출고수량'] > cond1) | (df['출고수량'] < cond2)])

(2102, 726)

In [ ]:
index1 = df[df['입수'] >= cond].index
index2 = df[(df['출고수량'] > cond1) | (df['출고수량'] < cond2)].index
df.shape , df.drop(index = set(index1) | set(index2)).shape
df2 = df.drop(index = set(index1) | set(index2))

In [ ]:
df.shape, df2.shape

((36833, 16), (34005, 16))

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34005 entries, 1 to 36832
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   날짜           34005 non-null  object 
 1   SKU ID       34005 non-null  int64  
 2   SKU 명        34005 non-null  object 
 3   바코드          34005 non-null  object 
 4   발주가능상태       34005 non-null  object 
 5   입고수량         34005 non-null  int64  
 6   쿠팡출고수량       34005 non-null  int64  
 7   쿠팡현재재고수량     34005 non-null  int64  
 8   자재번호         34005 non-null  int64  
 9   자재명          34005 non-null  object 
 10  카테고리         34005 non-null  object 
 11  입수           34005 non-null  int64  
 12  출고수량         34005 non-null  float64
 13  재고수량         34005 non-null  float64
 14  재고수량_scaled  34005 non-null  float64
 15  Target       34005 non-null  int64  
dtypes: float64(3), int64(7), object(6)
memory usage: 4.4+ MB


In [ ]:
df2['SKU 명'].value_counts()

홈즈 에어후레쉬 옷장용 방향소취제 시원한 아쿠아향 32g      84
2080애플민트향치약(자일리톨) 100gx3입            84
리큐진한겔드럼 2.1L                         84
순샘구연산 자몽 1.4L 리필                     84
리큐 반만쓰는 진한겔 드럼 2.4L                  84
                                     ..
Pack_샤워메이트 에블 바디 필링 티슈 브라이트닝 10매      7
Box_스파크 스팀100°C 1.4kg 기획 3개입          7
샤워메이트스크럽 300g쿠키앤크림                    7
샤워메이트스크럽 300g피스타치오                    7
Box_리큐 진한겔 액체세제 일반용 리필형 2.1L * 4입     4
Name: SKU 명, Length: 492, dtype: int64

In [ ]:
df2['자재명'].value_counts()

19리큐 진한겔 알카2.7용기(일)        211
18울샴푸오리지 1.8L (화)          168
16트리오투명한 생각 1000 mL 파우치    168
19리큐 진한겔 1/2 2.4용기(드)      168
17항균트리오 3kg(1종)            168
                          ... 
18 스파크 스팀100℃ 1.4kg 기획       7
19샤워메이트스크럽 300쿠키앤크림책         7
18샤워메이트스크럽 300피스타치오          7
17샤워메이트필링티슈 10매브라이트닝         7
17리큐 진한겔 2.1L온라인4입(화)        4
Name: 자재명, Length: 328, dtype: int64

In [ ]:
df3 = df2.drop(['SKU ID', '바코드', '자재번호', '재고수량'], axis = 1)

In [ ]:
#해야할것
#1. 날짜 분리
#2. 레이블 인코딩 자재명, SKU명
#3. 카테고리, 발주가능상태 원핫 인코딩
#4. 입고수량, 쿠팡출고수량, 쿠팡재고수량 스케일링

In [ ]:
df3.head(2)

,날짜,SKU 명,발주가능상태,입고수량,쿠팡출고수량,쿠팡현재재고수량,자재명,카테고리,입수,출고수량,재고수량_scaled,Target
1,2019-05-16,리큐 진한겔드럼 3L NEW,발주가능,0,235,2772,19리큐 진한겔 알카2.7용기(드),세탁세제,4,58.750000,-0.169367,0
2,2019-05-16,울샴푸아웃도어 800ml 용기,발주가능,0,2,35,19울샴푸아웃도 800ml(리뉴얼),세탁세제,9,0.222222,-0.092420,0


In [ ]:
#1.날짜분리
df3['날짜'] = pd.to_datetime(df3['날짜'])
df3['년도'] = df3['날짜'].dt.year
df3['월'] = df3['날짜'].dt.month
df3['일'] = df3['날짜'].dt.day
df3.head(3)

,날짜,SKU 명,발주가능상태,입고수량,쿠팡출고수량,쿠팡현재재고수량,자재명,카테고리,입수,출고수량,재고수량_scaled,Target,년도,월,일
1,2019-05-16,리큐 진한겔드럼 3L NEW,발주가능,0,235,2772,19리큐 진한겔 알카2.7용기(드),세탁세제,4,58.750000,-0.169367,0,2019,5,16
2,2019-05-16,울샴푸아웃도어 800ml 용기,발주가능,0,2,35,19울샴푸아웃도 800ml(리뉴얼),세탁세제,9,0.222222,-0.092420,0,2019,5,16
3,2019-05-16,울샴푸아웃도어 1L 일자형 리필,발주가능,207,41,2083,19울샴푸아웃도 1L(리뉴얼),세탁세제,9,4.555556,0.116655,0,2019,5,16


In [ ]:
df4 = df3.drop('날짜', axis = 1)
df4.head(2)

,SKU 명,발주가능상태,입고수량,쿠팡출고수량,쿠팡현재재고수량,자재명,카테고리,입수,출고수량,재고수량_scaled,Target,년도,월,일
1,리큐 진한겔드럼 3L NEW,발주가능,0,235,2772,19리큐 진한겔 알카2.7용기(드),세탁세제,4,58.750000,-0.169367,0,2019,5,16
2,울샴푸아웃도어 800ml 용기,발주가능,0,2,35,19울샴푸아웃도 800ml(리뉴얼),세탁세제,9,0.222222,-0.092420,0,2019,5,16


In [ ]:
#2. 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df4['SKU 명'] = le.fit_transform(df4['SKU 명'])
df4['자재명'] = le.fit_transform(df4['자재명'])
df4['카테고리'] = le.fit_transform(df4['카테고리'])

In [ ]:
df4.head()

,SKU 명,발주가능상태,입고수량,쿠팡출고수량,쿠팡현재재고수량,자재명,카테고리,입수,출고수량,재고수량_scaled,Target,년도,월,일
1,298,발주가능,0,235,2772,292,5,4,58.750000,-0.169367,0,2019,5,16
2,389,발주가능,0,2,35,317,5,9,0.222222,-0.092420,0,2019,5,16
3,388,발주가능,207,41,2083,316,5,9,4.555556,0.116655,0,2019,5,16
4,224,발주가능,288,312,3411,99,3,8,39.000000,-0.141272,0,2019,5,16
5,425,발주가능,0,0,86,277,7,10,0.000000,-0.202277,0,2019,5,16


In [ ]:
#3. 원핫 인코딩
df5 = pd.concat([df4,pd.get_dummies(df4['발주가능상태'])], axis = 1)
df5.drop('발주가능상태', axis = 1, inplace = True)
df5.head(2)

,SKU 명,입고수량,쿠팡출고수량,쿠팡현재재고수량,자재명,카테고리,입수,출고수량,재고수량_scaled,Target,년도,월,일,단종,발주가능,발주중단
1,298,0,235,2772,292,5,4,58.750000,-0.169367,0,2019,5,16,0,1,0
2,389,0,2,35,317,5,9,0.222222,-0.092420,0,2019,5,16,0,1,0


In [ ]:
#4. min max scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df6 = df5.copy()
df6[['입고수량', '쿠팡출고수량', '쿠팡현재재고수량','출고수량']] = scaler.fit_transform(df6[['입고수량', '쿠팡출고수량', '쿠팡현재재고수량','출고수량']])
df6.head()

,SKU 명,입고수량,쿠팡출고수량,쿠팡현재재고수량,자재명,카테고리,입수,출고수량,재고수량_scaled,Target,년도,월,일,단종,발주가능,발주중단
1,298,0.000000,0.304010,0.085765,292,5,4,0.896186,-0.169367,0,2019,5,16,0,1,0
2,389,0.000000,0.002587,0.001083,317,5,9,0.003390,-0.092420,0,2019,5,16,0,1,0
3,388,0.029337,0.053040,0.064447,316,5,9,0.069492,0.116655,0,2019,5,16,0,1,0
4,224,0.040816,0.403622,0.105535,99,3,8,0.594915,-0.141272,0,2019,5,16,0,1,0
5,425,0.000000,0.000000,0.002661,277,7,10,0.000000,-0.202277,0,2019,5,16,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split
X = df6.drop('Target', axis = 1)
y = df6['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((23803, 15), (10202, 15), (23803,), (10202,))

In [ ]:
#모델링 및 학습

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
rf = RandomForestClassifier()
param_grid = {'n_estimators' : [50, 100,200, 300],
              'max_depth' : [None, 3, 5, 7]}

gs = GridSearchCV(estimator=rf,
                  param_grid = param_grid,
                  n_jobs = -1,
                  cv = 5)
gs.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [None, 3, 5, 7],
                         'n_estimators': [50, 100, 200, 300]})

In [ ]:
best_rf = gs.best_estimator_
pred = best_rf.predict(X_test)

In [ ]:
pred_proba = best_rf.predict_proba(X_test)[:, 1]

In [ ]:
roc_auc_score(y_test,pred_proba)

1.0